The `lhs` operator, which we call $A$, is the product of the transpose of the first-order operator ($L^T$) and the first-order operator itself ($L$).

---
## The Operator Expression: A = LᵀL

The full expression for the `lhs` operator is the matrix product:

$$A = L^T L$$

This operation takes the system of first-order equations and effectively squares it, resulting in a robust, symmetric, second-order system.

---
## Matrix Components

To see the full expression, we can write out the matrices for both $L^T$ and $L$.

The **linearized first-order operator, L**, is:
$
L =
\begin{pmatrix}
\frac{\partial}{\partial t} + \mathbf{u}_k \cdot \nabla & 0 & \frac{\partial}{\partial x} & \frac{1}{Re} \frac{\partial}{\partial y} \\[1em]
0 & \frac{\partial}{\partial t} + \mathbf{u}_k \cdot \nabla & \frac{\partial}{\partial y} & -\frac{1}{Re} \frac{\partial}{\partial x} \\[1em]
\frac{\partial}{\partial x} & \frac{\partial}{\partial y} & 0 & 0 \\[1em]
\frac{\partial}{\partial y} & -\frac{\partial}{\partial x} & 0 & 1
\end{pmatrix}
The **transpose operator, $L^t$**, is its formal adjoint. This involves transposing the matrix and taking the adjoint of each differential operator (e.g., the adjoint of $\frac{\partial}{\partial x}$ is $-\frac{\partial}{\partial x}$):
$
L^T =
\begin{pmatrix}
(\frac{\partial}{\partial t} + \mathbf{u}_k \cdot \nabla)^T & 0 & -\frac{\partial}{\partial x} & -\frac{\partial}{\partial y} \\[1em]
0 & (\frac{\partial}{\partial t} + \mathbf{u}_k \cdot \nabla)^T & -\frac{\partial}{\partial y} & \frac{\partial}{\partial x} \\[1em]
-\frac{\partial}{\partial x} & -\frac{\partial}{\partial y} & 0 & 0 \\[1em]
-\frac{1}{Re}\frac{\partial}{\partial y} & \frac{1}{Re}\frac{\partial}{\partial x} & 0 & 1
\end{pmatrix}
$
The `lhs` operator $A$ is the 4x4 matrix that results from multiplying $L^T$ and $L$. The resulting terms are complex second-order differential operators. For example, the top-left entry of $A$ involves terms like $\frac{\partial^2}{\partial x^2}$ and $\frac{\partial^2}{\partial y^2}$.

---
## What This Means for the Code

Your code wisely avoids ever calculating the final, complicated expression for $A$. Instead, it applies the operators **sequentially**, which is the essence of a "matrix-free" method:
1.  It first computes an intermediate result by applying $L$ (this is Stage 3, forming the `su` array).
2.  It then computes the final result by applying $L^T$ to that intermediate result (this is Stage 4).

This sequential application is computationally much more efficient and simpler to program than deriving and coding the final, complex form of the $A$ operator. 🏗️

For completeness, here is the expression for the **right-hand side (RHS)** of the linear system.

The `rhs` of the equation $A \cdot x = b$ is the vector **b**, which contains all the known quantities from previous time steps and physical source terms.

---
## The Right-Hand Side Vector (b)

This vector is assembled from the BDF2 time-stepping formula using the known, converged solutions from the previous two time steps, `n` and `n-1`.

$$
\mathbf{b} =
\begin{pmatrix}
\frac{2u^n - 0.5u^{n-1}}{\Delta t} + S_u^{n+1} \\[1em]
\frac{2v^n - 0.5v^{n-1}}{\Delta t} + S_v^{n+1} \\[1em]
0 \\[1em]
0
\end{pmatrix}
$$

---
## Role in the `rhs` Subroutine

Your Fortran `rhs` subroutine calculates the full **residual vector** needed to drive the iterative solver. This residual is defined as $r_k = b - A \cdot x_k$, where $x_k$ is the current guess for the solution.

Using the operator notation we've established ($A = L^T L$), the residual vector calculated by the subroutine is:

$$\mathbf{r}_k = L^T \mathbf{b} - (L^T L) \mathbf{U}_k$$

This residual $\mathbf{r}_k$ is the actual right-hand side given to the `bicgstab` solver, which then finds the update $\Delta \mathbf{U}$ by solving the system $A \cdot \Delta \mathbf{U} = \mathbf{r}_k$. ✅

`Su` and `Sv` represent the **residuals of the x-momentum and y-momentum equations**, respectively.

In the context of the code, they are the first two components of the temporary `su` array. They measure how well the current solution guess satisfies the conservation of momentum in the x and y directions.

***
## The Equations

### Su (x-Momentum Residual)

This is the mathematical expression for the first component of the `su` array. It is the fully discretized and linearized x-momentum equation, rearranged so that all terms are on one side.

$$S_u = \underbrace{\left( \frac{1.5u_{k+1}^{n+1}}{\Delta t} \right)}_{\text{Time Derivative}} + \underbrace{\left( u_k^{n+1} \frac{\partial u_{k+1}^{n+1}}{\partial x} + v_k^{n+1} \frac{\partial u_{k+1}^{n+1}}{\partial y} \right)}_{\text{Convection}} + \underbrace{\frac{\partial p_{k+1}^{n+1}}{\partial x}}_{\text{Pressure Gradient}} + \underbrace{\frac{1}{Re} \frac{\partial \omega_{k+1}^{n+1}}{\partial y}}_{\text{Viscous Term}} - \underbrace{\left( \frac{2u^n - 0.5u^{n-1}}{\Delta t} \right)}_{\text{Past Time Steps}}$$

### Sv (y-Momentum Residual)

Similarly, this is the expression for the second component of the `su` array, representing the y-momentum equation.

$$S_v = \underbrace{\left( \frac{1.5v_{k+1}^{n+1}}{\Delta t} \right)}_{\text{Time Derivative}} + \underbrace{\left( u_k^{n+1} \frac{\partial v_{k+1}^{n+1}}{\partial x} + v_k^{n+1} \frac{\partial v_{k+1}^{n+1}}{\partial y} \right)}_{\text{Convection}} + \underbrace{\frac{\partial p_{k+1}^{n+1}}{\partial y}}_{\text{Pressure Gradient}} - \underbrace{\frac{1}{Re} \frac{\partial \omega_{k+1}^{n+1}}{\partial x}}_{\text{Viscous Term}} - \underbrace{\left( \frac{2v^n - 0.5v^{n-1}}{\Delta t} \right)}_{\text{Past Time Steps}}$$

The Least-Squares method takes these two residuals—along with the residuals for continuity and vorticity—and minimizes their squared values to find the final solution at the new time step. 🌊

Of course. Here is the Jacobian matrix, $J$, for the fully coupled $u,v,p,\omega$ system.

The entries in this matrix are **differential operators**. After discretization, these operators become the coefficients in the large, sparse matrix that is solved at each Newton iteration.

***

### The Jacobian Matrix  J  MATRIX

$
J =
\begin{bmatrix}
\left( u \frac{\partial}{\partial x} + v \frac{\partial}{\partial y} \right) + \frac{\partial u}{\partial x} & \frac{\partial u}{\partial y} & \frac{1}{\rho}\frac{\partial}{\partial x} & \nu\frac{\partial}{\partial y} \\
\\
\frac{\partial v}{\partial x} & \left( u \frac{\partial}{\partial x} + v \frac{\partial}{\partial y} \right) + \frac{\partial v}{\partial y} & \frac{1}{\rho}\frac{\partial}{\partial y} & -\nu\frac{\partial}{\partial x} \\
\\
\frac{\partial}{\partial x} & \frac{\partial}{\partial y} & 0 & 0 \\
\\
\frac{\partial}{\partial y} & -\frac{\partial}{\partial x} & 0 & 1
\end{bmatrix}
$

***

### Breakdown of the Terms 🔬

* **Top-Left 2x2 Block**: This section represents the linearization of the non-linear **convective terms** from the momentum equations. The diagonal terms include the convective operator itself, while the off-diagonal terms show how the momentum in one direction is affected by the velocity in the other.
* **Third Column (Pressure)**: These are the **pressure gradient** operators, linking the momentum equations to pressure. Note the zero in the third and fourth rows, as pressure does not appear in the continuity or vorticity-definition equations.
* **Fourth Column (Vorticity)**: These terms represent the **viscous coupling**. They show how gradients in vorticity ($\omega$) influence the momentum equations ($R_u$ and $R_v$). The '1' in the bottom right corner comes directly from the $\omega$ term in the residual $R_{\omega} = \omega - (\dots)$.
* **Third and Fourth Rows**: These represent the linearized **continuity** and **vorticity definition** equations, which are already linear. They form constraints that couple the velocity field components to each other and to the vorticity field.


They are the **residuals** of the governing equations. A residual is what's left over when you plug an approximate solution into an equation that should ideally equal zero. It's a direct measure of the error in your current guess.

Think of it like trying to balance a scale. If the two sides aren't perfectly equal, the difference between them is the residual. The entire goal of the solver (`bicgstab`) is to adjust the solution variables ($u, v, p, \omega$) until these residuals are driven as close to zero as possible, meaning the equations are satisfied and the "scale" is balanced. ✅

The linear system is a large, sparse matrix equation that takes the following block form:

$$
\begin{bmatrix}
\ddots & & & & \\
& J_{i, i-1} & J_{i,i} & J_{i, i+1} & \\
& & & & \ddots
\end{bmatrix}
\begin{bmatrix}
\vdots \\
\delta X_i \\
\vdots
\end{bmatrix}
=
\begin{bmatrix}
\vdots \\
-F_i \\
\vdots
\end{bmatrix}
$$This equation, in the form $A\\vec{x} = \\vec{b}$, is what a computer solves at every single step of the Newton method. Let's break down what each part looks like.

-----

### From Operators to Numbers: Discretization

The Jacobian you saw before was a matrix of *differential operators* (like $\frac{\partial}{\partial x}$). To build the actual numerical system, we replace these operators with their **discrete approximations**, a process called **discretization**.

For example, using a simple finite difference scheme, the operator $\frac{\partial}{\partial x}$ acting on a variable at grid point `i` becomes a numerical **stencil**. A second-order central difference would be:

$\frac{\partial f_i}{\partial x} \approx \frac{f_{i+1} - f_{i-1}}{2 \Delta x}$

This means the operator is replaced by numbers that couple grid point `i` to its neighbors (`i+1` and `i-1`).

-----

### The System Matrix (Global Jacobian) 🖥️

When we apply this discretization to every operator in the 4x4 Jacobian block, we get the global system matrix.

* **Structure**: It's a massive **block matrix**. Each element shown in the schematic above (e.g., $J\_{i,i}$) is itself a 4x4 matrix of numbers.
* **Diagonal Blocks ($J\_{i,i}$)**: These blocks represent how the four equations at a grid point `i` depend on the four variables at that *same* grid point `i`.
* **Off-Diagonal Blocks ($J\_{i, i\\pm1}$)**: These represent the influence of neighboring grid points. For example, $J\_{i, i+1}$ is a 4x4 matrix describing how the equations at point `i` depend on the variables at point `i+1`.
* **Sparsity**: Because the discrete derivatives only connect a point to its immediate neighbors, most of the blocks in the global matrix are zero. This makes the system **sparse**, which is crucial for solving it efficiently.

-----

### The Vectors (Unknowns and Residuals)

The vectors in the equation are long column vectors that stack the variables from every grid point in the domain.

* **Update Vector ($\delta X$)**: This is the vector of unknowns we are solving for. It contains the corrections for every variable at every point.

$
\delta X =
\begin{bmatrix}
\vdots \\
\hline
\delta u_i \\
\delta v_i \\
\delta p_i \\
\delta \omega_i \\
\hline
\delta u_{i+1} \\
\delta v_{i+1} \\
\delta p_{i+1} \\
\delta \omega_{i+1} \\
\hline
\vdots
\end{bmatrix}
$

  * **Residual Vector ($-F$)**: This is the "right-hand side" of the equation. It's a vector of known numbers calculated from the previous iteration's solution, representing how much the current solution fails to satisfy the governing equations.

    $
    -F =
    \begin{bmatrix}
    \vdots \\
    \hline
    -R_{u_i} \\
    -R_{v_i} \\
    -R_{c_i} \\
    -R_{\omega,i} \\
    \hline
    -R\_{u,i+1} \\
    -R\_{v,i+1} \\
    -R\_{c,i+1} \\
    -R\_{\omega,i+1} \\
    \hline
    \vdots
    \end{bmatrix}
    $

   In summary, the abstract Newton step $J \delta X = -F$ becomes a concrete, massive but highly structured system of linear algebraic equations. The sheer size of this system is why solving it efficiently with specialized **iterative solvers** (like GMRES) is the most computationally expensive part of the entire simulation.